In [1]:
import os
import torch
import numpy as np
import pickle
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tokenizer import Tokenizer
from generation_manager import GeneratorManager

In [2]:
tokenizer = Tokenizer()
print('tokenizer loaded')

curr_model_path = '/home/saitto/progen/ckpt/_epoch3_chunk10.pth' # model checkpoint
print('loading model from:', curr_model_path)
# Saving only state_dict:
#torch.save(model.state_dict(), model_state_dict_path)
#model_state_dict_path = load_model_path + 'model_only_state_dict_v0Last.pth'
#print('Model state dict saved')

tokenizer loaded
loading model from: /home/saitto/progen/ckpt/_epoch3_chunk10.pth


In [13]:
# paths to the saved .p files
#query = "PF00959"
# /home/saitto/progen/ProGen/my_data/test.p
#random_selection_file = os.path.join("data", "random_selection_2000_" + query + ".p")
#data_file = os.path.join("data", "filtered_data_" + query + ".p")

data_file = '/home/saitto/progen/ProGen/my_data/test.p'

# Reload random_selection from the .p file
with open(random_selection_file, "rb") as file:
    random_selection = pickle.load(file)

# Reload filtered_data from the .p file
#filtered_data = False
#with open(data_file, "rb") as file:
#    filtered_data = pickle.load(file)

# Print or use the reloaded datasets as needed
print("Random Selection:")
for entry in random_selection:
    print(entry["sequence"])
    print(len(random_selection))
    break

#print("---")

#print("Filtered Data:")
#for entry in filtered_data:
#    print(entry["sequence"])

Random Selection:
{'metadata': {'accession': 'A0A376DZW2', 'id': 'A0A376DZW2_9ENTR', 'source_organism': {'taxId': '1074000', 'scientificName': 'Cronobacter universalis NCTC 9529', 'fullName': 'Cronobacter universalis NCTC 9529'}, 'name': 'Lysozyme', 'description': None, 'length': 150, 'sequence': 'MANIPTNTGMPGVELIKSFEGLRLDKYQDAVGKWTIGYGHLILPNENFTRALTLQEAEDLLRADLGMTERGIRQMVKVDLNQNQFDALVSFAFNVGLGNLQSSTLLRLLNQGSYREAADQLLRWNKAGGNVLAGLTRRREAERLLFLTAL', 'proteome': None, 'gene': 'NCTC9529_00542', 'go_terms': [{'identifier': 'GO:0003796', 'name': 'lysozyme activity', 'category': {'code': 'F', 'name': 'molecular_function'}}, {'identifier': 'GO:0009253', 'name': 'peptidoglycan catabolic process', 'category': {'code': 'P', 'name': 'biological_process'}}, {'identifier': 'GO:0016998', 'name': 'cell wall macromolecule catabolic process', 'category': {'code': 'P', 'name': 'biological_process'}}], 'protein_evidence': 3, 'source_database': 'unreviewed', 'is_fragment': False, 'ida_accession': 'b8801d01

In [4]:
'''query = "PF00959"
random_selection_file = os.path.join("data", "sampled_data_validation_2000_" + query + ".p")
data_file = os.path.join("data", "filtered_data_" + query + ".p")

# Reload random_selection from the .p file
random_selection = False
with open(random_selection_file, "rb") as file:
    random_selection = pickle.load(file)

# Reload filtered_data from the .p file
#filtered_data = False
#with open(data_file, "rb") as file:
#    filtered_data = pickle.load(file)

# Print or use the reloaded datasets as needed
print("Random Selection:")
#random_selection = random_selection[0:100]
for entry in random_selection:
    print(entry["sequence"])
    print(len(random_selection))
    break'''


'query = "PF00959"\nrandom_selection_file = os.path.join("data", "sampled_data_validation_2000_" + query + ".p")\ndata_file = os.path.join("data", "filtered_data_" + query + ".p")\n\n# Reload random_selection from the .p file\nrandom_selection = False\nwith open(random_selection_file, "rb") as file:\n    random_selection = pickle.load(file)\n\n# Reload filtered_data from the .p file\n#filtered_data = False\n#with open(data_file, "rb") as file:\n#    filtered_data = pickle.load(file)\n\n# Print or use the reloaded datasets as needed\nprint("Random Selection:")\n#random_selection = random_selection[0:100]\nfor entry in random_selection:\n    print(entry["sequence"])\n    print(len(random_selection))\n    break'

In [5]:
import ete3
ncbi = ete3.NCBITaxa()
# to update the taxonomic local database use:
# ncbi.update_taxonomy_database()
'''taxon_id = 505
lineage = ncbi.get_lineage(taxon_id)
print(lineage)
print(type(lineage))'''

'taxon_id = 505\nlineage = ncbi.get_lineage(taxon_id)\nprint(lineage)\nprint(type(lineage))'

In [6]:
count = 0
inner_count = 0
for entry in random_selection:
    taxid = int(entry['metadata']['source_organism']['taxId']) # taxonomy id from NCBI
    inner_count = 0
    #print(taxid)
    try:
        tax_lineage1 = ncbi.get_lineage(taxid)
        # remove root taxonomy id
        tax_lineage1 = [element for element in tax_lineage1 if element not in [1, 2, 131567, 10239]]
        #tax_lineage2 = tokenizer.taxa_to_lineage[taxid] # make lineage in ncbi ids
        #print(tax_lineage1)
        res = []
        for ite in tax_lineage1:
            try:
                tmp = tokenizer.taxa_to_ctrl_idx[ite]
                res.append(tmp)
            except:
                inner_count += 1
        if res == [] and inner_count >= 1:
            count += 1
    except:
        count += 1
        print("FATAL taxa conversion")
        #tax_lineage = [tokenizer.taxa_to_ctrl_idx[ite] for ite in tax_lineage1] # now translated as ctrl code indices
print("Tax lineage problems:", count)
    # break

Tax lineage problems: 84


<p>To evaluate the generated sequence using teacher forcing:

Generate the Sequence: The transformer model with teacher forcing generates the complete sequence. 
At each time step, we provide the true input sequence (+ 1 versus previous step), as input to the model.

</p>

In [7]:
penalty = 0
topk = 1
generator = GeneratorManager(curr_model_path, penalty=penalty, topk=topk)

predicted = []
true_tokens_index_in_probs_all = []
tokens_probs_all = []
true_value = []
for entry in random_selection:
    input_seq = entry['sequence']['metadata']['sequence']
    #print(input_seq)
    '''
    taxid = int(entry['metadata']['source_organism']['taxId']) # taxonomy id from NCBI
    try:
        tax_lineage = ncbi.get_lineage(taxid)
        # remove root taxonomy id
        tax_lineage = [element for element in tax_lineage1 if element not in [1, 2, 131567, 10239]]
        res = []
        inner_count = 0
        for ite in tax_lineage:
            try:
                tmp = tokenizer.taxa_to_ctrl_idx[ite]
                res.append(tmp)
            except:
                inner_count += 1
        if res == [] and inner_count >= 1:
            print("ALERT: error in taxonmy conversions for entry: ", taxid)
        tax_lineage = res
    except:
        print("FATAL taxa conversion for entry: ", taxid)
        # tax_lineage = tokenizer.taxa_to_lineage[taxid] # make lineage in ncbi ids
        # tax_lineage = [tokenizer.taxa_to_ctrl_idx[ite] for ite in tax_lineage] # now translated as ctrl code indices
    '''
    # FOR TESTIG the code:
    # input_seq = input_seq[0:24]
    
    # if no tax keys:
    tax_lineage = [0]
    #print(input_seq)
    
    #try:
    # res, tokens_prob = teacher_forcing_generation(input_seq, tax_lineage)
    res, tokens_prob, offset = generator.after_n_generation(input_seq, tax_lineage, 20)
    #except:
    #    print("ALERT: skipped for error the entry: ", entry['metadata']['accession'])
    #    continue
    # print("Input: ", input_seq)
    # print("Res: ", res)
    true_value.append(input_seq)
    true_tokens_index_in_prob = []
    true_tokens_index_in_probs_all.append(true_tokens_index_in_prob)
    tokens_probs_all.append(tokens_prob)
    predicted.append(input_seq[:offset] + res)

    # FOR TESTIG:
    break
    
    

MODEL SIZE: 
1280
Found PyTorch checkpoint at  /home/saitto/progen/ckpt/_epoch3_chunk10.pth
GPU aviable. Previous checkpoint loaded in GPU


In [11]:
print(len(predicted))

1


In [12]:
print(predicted)

['MANIPTNTGMPGVELIKSFEELTFTLSHAGSGSPIVFLHGNPTSSYLWRNIIPHVSDLGRCLAPDLIGMGESGKPDIDYTFADHARYLDAFIEALGLERVTLVLHDWGSALGFHWAHRHPERVKAIAYMEAIVRPVTWDEWPEAARGIFQ']


In [10]:
# Create a directory named "ID_test_data" in the current working directory if it doesn't exist
data_dir = "PF00959_generalModel_after_50_proteins_2000_p_0_k_1_key_only_0"
query = "PF00959"
os.makedirs(data_dir, exist_ok=True)

# TODO, salvare tokens_prob e true_tokens_index_in_prob
tokens_probs_data_file = os.path.join(data_dir, "tokens_probs_data_" + query + ".p")
with open(tokens_probs_data_file, "wb") as file:
    pickle.dump(tokens_probs_all, file)
    
true_tokens_index_data_file = os.path.join(data_dir, "true_tokens_index_data_" + query + ".p")
with open(true_tokens_index_data_file, "wb") as file:
    pickle.dump(true_tokens_index_in_probs_all, file)

# Save ID_test_data as a .p file
predicted_data_file = os.path.join(data_dir, "predicted_data_" + query + ".p")
with open(predicted_data_file, "wb") as file:
    pickle.dump(predicted, file)
    
# Save ID_test_data as a .p file
true_data_file = os.path.join(data_dir, "true_data_" + query + ".p")
with open(true_data_file, "wb") as file:
    pickle.dump(true_value, file)
torch.cuda.empty_cache()